# **Data Preprocessing**

# Introduction

In this notebook, we'll run the first approach to data processing, where we'll get to know the dataset and focus on scope definition, feature selection, and data formatting. Exploratory Data Analysis will be done in the next phase (another notebook). 

The main process will include the following steps:

1. Filter out records out of scope:

- Exclude ages between 0 and 2 years, as in México, formal education begins at age 3.
- Ensure that the person has been enrolled in school at some point.
- Remove records without labels.

2. Keep features related to the previous school cycle, as this is the part of the dataset that contains labels.

- It is not useful to include the current cycle, as no related labels are available.

3. Format the dataset in a useful and standard way:

- The current dataset contains incorrect data types, mixed encoding, and inconsistent value selection.
- In the source dataset, all features are stored as alphanumeric values. Selected features need to be converted into their correct format to align with analysis and modeling requirements.
- For features representing combinations of scenarios (e.g., a student having both a laptop and a smartphone), it is crucial to retain their separate representation across columns, as merging them into a single column could lead to loss of granularity and interpretability.

Considerations:

- We will use the DictVectorizer class from the scikit-learn library, so it is important to understand how it works. According to its documentation (https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.DictVectorizer.html), this class applies encoding only to features or columns that are of type string. Therefore, if we have categorical variables represented numerically, it will be essential to ensure they are handled as text.

- Additionally, we will need to make feature-based decisions, as some features are unique to an instance (in this case, a student). For example, school grade is typically a single value because a student is usually enrolled in one level at a time. In contrast, there are other features, like support devices, where a student may have multiple options simultaneously, such as a laptop, smartphone, and so on. These differences must be accounted for to ensure accurate representation and processing of the dataset.

- The primary premises will be to maintain transparency, interpretability, and avoid unnecessary data processing, reducing computational loads as early as possible in the lifecycle.

- we will adhere to the convention of using lowercase letters and replacing spaces with underscores for categorical values. This ensures consistency, minimizes errors during data processing, and enhances compatibility with tools and scripts. By following this format, we aim to streamline data manipulation and maintain a clear and uniform structure throughout the analysis.

# Dataset Preparation

Import pandas and file as a dataframe

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('enape_raw_dataset.csv')

In [3]:
df.head()

,FOLIO,N_REN,SEXO,EDAD,P3_1,P3_2,PA3_1,PA3_2,PA3_3_NIVEL,PA3_3_BIMESTRE,...,PD3_1,PD3_2,PD3_3,ENT,NIVEL_A,GRADO_A,NIVEL_B,GRADO_B,ESC,FACTOR
0,2,1,1,13,2,NaN,1.0,1.0,3.0,NaN,...,NaN,NaN,NaN,24,3.0,6.0,3.0,1.0,2.0,809
1,2,2,2,19,2,NaN,1.0,1.0,9.0,NaN,...,7.0,NaN,NaN,24,9.0,1.0,8.0,2.0,5.0,809
2,7,1,2,8,2,NaN,1.0,1.0,3.0,NaN,...,NaN,NaN,NaN,22,3.0,2.0,2.0,3.0,2.0,722
3,7,2,1,26,7,NaN,2.0,NaN,NaN,NaN,...,1.0,40.0,8.0,22,NaN,NaN,NaN,NaN,4.0,722
4,9,1,1,27,1,NaN,1.0,2.0,9.0,NaN,...,1.0,25.0,8.0,26,9.0,1.0,8.0,2.0,5.0,425


Keep only records between 3 and 29

Initial records count

In [4]:
df.shape[0]

32343

Filter ages below 3 years

In [5]:
df = df[df['EDAD'] > 2]

Check new count

In [6]:
df.shape[0]

30288

Feature PA3_1 indicates it the student was enrolled in last school cycle.

We'll keep students that were enrolled (value = 1)

In [7]:
df = df[df['PA3_1'] == 1 ]

In [8]:
df.shape[0]

20064

Feature PA3_4 indicates whether the student completed the last school cycle.

We will keep only labeled records (non-null values).

Remember, this survey is answered by a family member living in the same household as the student. Therefore, it is possible that this question was not answered or is null due to the sequence of previous questions.

In [9]:
df['PA3_4'].isnull().sum()

np.int64(91)

In [10]:
df = df[df['PA3_4'].notnull()]

In [11]:
df.shape[0]

19973

## Standarize format

### Ensure correct type for categorical columns

**Sex**

1: Male
2: Female

In [12]:
df['SEXO'].isnull().sum()

np.int64(0)

In [13]:
df.groupby('SEXO').size()

SEXO
1    10231
2     9742
dtype: int64

In [14]:
# Replace values in PA3_2
df['SEXO'] = df['SEXO'].replace({1: 'male', 2: 'female'})

# Rename the column
df.rename(columns={'SEXO': 'sex'}, inplace=True)

In [15]:
df.groupby('sex').size()

sex
female     9742
male      10231
dtype: int64

**School Type**

The PA3_2 value indicates the type of school: 1 represents public, and 2 represents private.
We will replace these values and rename the column to enhance interpretability.


Initial check

In [16]:
df.shape[0]

19973

In [17]:
df.head()

,FOLIO,N_REN,sex,EDAD,P3_1,P3_2,PA3_1,PA3_2,PA3_3_NIVEL,PA3_3_BIMESTRE,...,PD3_1,PD3_2,PD3_3,ENT,NIVEL_A,GRADO_A,NIVEL_B,GRADO_B,ESC,FACTOR
0,2,1,male,13,2,NaN,1.0,1.0,3.0,NaN,...,NaN,NaN,NaN,24,3.0,6.0,3.0,1.0,2.0,809
1,2,2,female,19,2,NaN,1.0,1.0,9.0,NaN,...,7.0,NaN,NaN,24,9.0,1.0,8.0,2.0,5.0,809
2,7,1,female,8,2,NaN,1.0,1.0,3.0,NaN,...,NaN,NaN,NaN,22,3.0,2.0,2.0,3.0,2.0,722
4,9,1,male,27,1,NaN,1.0,2.0,9.0,NaN,...,1.0,25.0,8.0,26,9.0,1.0,8.0,2.0,5.0,425
7,14,2,male,11,1,NaN,1.0,2.0,3.0,NaN,...,NaN,NaN,NaN,18,3.0,5.0,2.0,6.0,2.0,413


In [18]:
df['PA3_2'].isnull().sum()

np.int64(0)

In [19]:
df.groupby('PA3_2').size()

PA3_2
1.0    17826
2.0     2147
dtype: int64

Replace and rename

In [20]:
# Replace values in PA3_2
df['PA3_2'] = df['PA3_2'].replace({1: 'public', 2: 'private'})

# Rename the column
df.rename(columns={'PA3_2': 'school_type'}, inplace=True)

In [21]:
df.groupby('school_type').size()

school_type
private     2147
public     17826
dtype: int64

**Grade**

The PA3_3_NIVEL indicates the school grade in which student was registered last cycle.

In this case we'll replace numerical values with descriptive codes, to enhance interpretability an simplify model by standarizing formats

| **Numeric Value** | **Description (In Spanish)**                                             | **Description (Translated to English)**                                         | **New Code**    |
|-------------------|----------------------------------------------------------------------|-------------------------------------------------------------------------------|------------------|
| 01                | Educación inicial (guardería)                                       | Initial Education (Daycare)                                                  | daycare          |
| 02                | Preescolar                                                          | Preschool                                                                     | preschool        |
| 03                | Primaria                                                            | Primary School                                                                | primary          |
| 04                | Secundaria                                                          | Secondary School                                                              | secondary        |
| 05                | Profesional técnico (carrera técnica con secundaria terminada)      | Technical Professional (Technical degree with completed secondary school)     | tech_school      |
| 06                | Preparatoria o bachillerato general                                 | High School or General Baccalaureate                                          | high_school      |
| 07                | Bachillerato tecnológico                                            | Technological Baccalaureate                                                  | tech_bacc        |
| 08                | Técnico Superior Universitario (carrera técnica con bachillerato terminado) | Higher University Technician (Technical degree with completed high school) | univ_tech        |
| 09                | Licenciatura o profesional                                          | Bachelor's Degree or Professional Degree                                     | bachelors        |
| 10                | Especialidad                                                        | Specialization                                                               | specialty        |
| 11                | Maestría                                                            | Master's Degree                                                              | masters          |
| 12                | Doctorado                                                           | Doctorate                                                                    | doctorate        |


Initial check

In [22]:
df.shape[0]

19973

In [23]:
df.head()

,FOLIO,N_REN,sex,EDAD,P3_1,P3_2,PA3_1,school_type,PA3_3_NIVEL,PA3_3_BIMESTRE,...,PD3_1,PD3_2,PD3_3,ENT,NIVEL_A,GRADO_A,NIVEL_B,GRADO_B,ESC,FACTOR
0,2,1,male,13,2,NaN,1.0,public,3.0,NaN,...,NaN,NaN,NaN,24,3.0,6.0,3.0,1.0,2.0,809
1,2,2,female,19,2,NaN,1.0,public,9.0,NaN,...,7.0,NaN,NaN,24,9.0,1.0,8.0,2.0,5.0,809
2,7,1,female,8,2,NaN,1.0,public,3.0,NaN,...,NaN,NaN,NaN,22,3.0,2.0,2.0,3.0,2.0,722
4,9,1,male,27,1,NaN,1.0,private,9.0,NaN,...,1.0,25.0,8.0,26,9.0,1.0,8.0,2.0,5.0,425
7,14,2,male,11,1,NaN,1.0,private,3.0,NaN,...,NaN,NaN,NaN,18,3.0,5.0,2.0,6.0,2.0,413


In [24]:
df['PA3_3_NIVEL'].isnull().sum()

np.int64(0)

In [25]:
df.groupby('PA3_3_NIVEL').size()

PA3_3_NIVEL
2.0     2199
3.0     7172
4.0     3694
5.0       44
6.0     2684
7.0      758
8.0       94
9.0     3224
10.0      17
11.0      80
12.0       7
dtype: int64

In [26]:
# new value dictionary
education_levels = {
    1: 'daycare',
    2: 'preschool',
    3: 'primary',
    4: 'secondary',
    5: 'tech_school',
    6: 'high_school',
    7: 'tech_bacc',
    8: 'univ_tech',
    9: 'bachelors',
    10: 'specialty',
    11: 'masters',
    12: 'doctorate'
}

# Replace values
df['PA3_3_NIVEL'] = df['PA3_3_NIVEL'].replace(education_levels)

# Rename the column
df.rename(columns={'PA3_3_NIVEL': 'school_grade'}, inplace=True)

post check

In [27]:
df.shape[0]

19973

In [28]:
df.head()

,FOLIO,N_REN,sex,EDAD,P3_1,P3_2,PA3_1,school_type,school_grade,PA3_3_BIMESTRE,...,PD3_1,PD3_2,PD3_3,ENT,NIVEL_A,GRADO_A,NIVEL_B,GRADO_B,ESC,FACTOR
0,2,1,male,13,2,NaN,1.0,public,primary,NaN,...,NaN,NaN,NaN,24,3.0,6.0,3.0,1.0,2.0,809
1,2,2,female,19,2,NaN,1.0,public,bachelors,NaN,...,7.0,NaN,NaN,24,9.0,1.0,8.0,2.0,5.0,809
2,7,1,female,8,2,NaN,1.0,public,primary,NaN,...,NaN,NaN,NaN,22,3.0,2.0,2.0,3.0,2.0,722
4,9,1,male,27,1,NaN,1.0,private,bachelors,NaN,...,1.0,25.0,8.0,26,9.0,1.0,8.0,2.0,5.0,425
7,14,2,male,11,1,NaN,1.0,private,primary,NaN,...,NaN,NaN,NaN,18,3.0,5.0,2.0,6.0,2.0,413


In [29]:
df.groupby('school_grade').size().sort_values(ascending=False)

school_grade
primary        7172
secondary      3694
bachelors      3224
high_school    2684
preschool      2199
tech_bacc       758
univ_tech        94
masters          80
tech_school      44
specialty        17
doctorate         7
dtype: int64

**Periodicity Type**

The following columns indicate the type of period used by the school where the student was enrolled and the specific period in which the student was enrolled. We will unify these columns for model simplicity, as we consider that no information is lost since the column does not contain any value that serves as a reference to determine how close or far the student is from completing the grade.

- PA3_3_BIMESTRE
- PA3_3_TRIMESTRE
- PA3_3_CUATRIMESTRE
- PA3_3_SEMESTRE
- PA3_3_ANIO
- PA3_3_MODMAT


Initial check

In [30]:
df.shape

(19973, 101)

In [31]:
df[['PA3_3_BIMESTRE','PA3_3_TRIMESTRE','PA3_3_CUATRIMESTRE','PA3_3_SEMESTRE','PA3_3_ANIO','PA3_3_MODMAT']].head(10)

,PA3_3_BIMESTRE,PA3_3_TRIMESTRE,PA3_3_CUATRIMESTRE,PA3_3_SEMESTRE,PA3_3_ANIO,PA3_3_MODMAT
0,NaN,NaN,NaN,NaN,6.0,NaN
1,NaN,NaN,NaN,NaN,1.0,NaN
2,NaN,NaN,NaN,NaN,2.0,NaN
4,NaN,NaN,2.0,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,5.0,NaN
8,NaN,NaN,NaN,NaN,1.0,NaN
9,NaN,NaN,NaN,4.0,NaN,NaN
10,NaN,NaN,NaN,NaN,1.0,NaN
12,NaN,NaN,NaN,1.0,NaN,NaN
13,NaN,NaN,NaN,NaN,2.0,NaN


In [32]:
# Create new columns
df['period_type'] = None
df['period_number'] = None

In [33]:
# identify period type and number
def identify_period(row):
    if not pd.isna(row['PA3_3_BIMESTRE']):
        return 'bimester', row['PA3_3_BIMESTRE']
    elif not pd.isna(row['PA3_3_TRIMESTRE']):
        return 'trimester', row['PA3_3_TRIMESTRE']
    elif not pd.isna(row['PA3_3_CUATRIMESTRE']):
        return 'quadrimester', row['PA3_3_CUATRIMESTRE']
    elif not pd.isna(row['PA3_3_SEMESTRE']):
        return 'semester', row['PA3_3_SEMESTRE']
    elif not pd.isna(row['PA3_3_ANIO']):
        return 'year', row['PA3_3_ANIO']
    elif not pd.isna(row['PA3_3_MODMAT']):
        return 'module', row['PA3_3_MODMAT']
    else:
        return None, None

# Aplicar la función a cada fila
df[['period_type', 'period_number']] = df.apply(
    lambda row: pd.Series(identify_period(row)), axis=1
)

In [34]:
df[['PA3_3_BIMESTRE','PA3_3_TRIMESTRE','PA3_3_CUATRIMESTRE','PA3_3_SEMESTRE','PA3_3_ANIO','PA3_3_MODMAT','period_type', 'period_number']].head(10)

,PA3_3_BIMESTRE,PA3_3_TRIMESTRE,PA3_3_CUATRIMESTRE,PA3_3_SEMESTRE,PA3_3_ANIO,PA3_3_MODMAT,period_type,period_number
0,NaN,NaN,NaN,NaN,6.0,NaN,year,6.0
1,NaN,NaN,NaN,NaN,1.0,NaN,year,1.0
2,NaN,NaN,NaN,NaN,2.0,NaN,year,2.0
4,NaN,NaN,2.0,NaN,NaN,NaN,quadrimester,2.0
7,NaN,NaN,NaN,NaN,5.0,NaN,year,5.0
8,NaN,NaN,NaN,NaN,1.0,NaN,year,1.0
9,NaN,NaN,NaN,4.0,NaN,NaN,semester,4.0
10,NaN,NaN,NaN,NaN,1.0,NaN,year,1.0
12,NaN,NaN,NaN,1.0,NaN,NaN,semester,1.0
13,NaN,NaN,NaN,NaN,2.0,NaN,year,2.0


In [35]:
df.shape

(19973, 103)

In [36]:
df.drop(columns=['PA3_3_BIMESTRE','PA3_3_TRIMESTRE','PA3_3_CUATRIMESTRE','PA3_3_SEMESTRE','PA3_3_ANIO','PA3_3_MODMAT'], inplace=True)


In [37]:
df.shape

(19973, 97)

In [38]:
df[['period_type', 'period_number']].head(10)

,period_type,period_number
0,year,6.0
1,year,1.0
2,year,2.0
4,quadrimester,2.0
7,year,5.0
8,year,1.0
9,semester,4.0
10,year,1.0
12,semester,1.0
13,year,2.0


In [39]:
df.groupby(['period_type', 'period_number']).size()


period_type   period_number
bimester      1.0                 2
              2.0                 3
              3.0                 3
              4.0                 2
              6.0                 3
              8.0                 2
              9.0                 1
module        9.0                36
quadrimester  1.0                33
              2.0                25
              3.0                62
              4.0                46
              5.0                27
              6.0                29
              7.0                21
              8.0                24
              9.0                27
              10.0               10
              11.0                3
              12.0                4
              15.0                1
semester      1.0               268
              2.0               805
              3.0               228
              4.0               817
              5.0               223
              6.0               627


**Expected grade**

The PB3_15 value indicates what the expected grade to achieve according to parents o guardian

In this case we'll replace numerical values with descriptive codes, to enhance interpretability an simplify model by standarizing formats

| Numeric Value | Description (in Spanish)                                  | Description (Translated)                              | New Value      |
|---------------|-----------------------------------------------------------|------------------------------------------------------|----------------|
| 1             | Primaria                                                  | Primary School                                       | primary        |
| 2             | Secundaria                                                | Secondary School                                     | secondary      |
| 3             | Carrera técnica con secundaria terminada (Profesional técnico) | Technical Career with Secondary Completed           | tech_school    |
| 4             | Preparatoria o Bachillerato general                       | General High School                                  | high_school    |
| 5             | Bachillerato tecnológico                                  | Technological High School                           | tech_bacc      |
| 6             | Carrera técnica con bachillerato terminado (Técnico Superior Universitario) | Technical Career with High School Completed        | univ_tech      |
| 7             | Licenciatura                                              | Bachelor's Degree                                    | bachelors      |
| 8             | Posgrado                                                  | Postgraduate (Master's or Doctorate)                | masters        |
| 9             | No sabe                                                   | Doesn't Know                                         | unknown           |


Initial check

In [40]:
df['PB3_15'].isnull().sum()

np.int64(5545)

In [41]:
df.groupby('PB3_15').size()

PB3_15
1.0       27
2.0      173
3.0       14
4.0      506
5.0      183
6.0      205
7.0    11187
8.0     1776
9.0      357
dtype: int64

In [42]:
# new value dictionary
expected_grd = {
    1: 'primary',
    2: 'secondary',
    3: 'tech_school',
    4: 'high_school',
    5: 'tech_bacc',
    6: 'univ_tech',
    7: 'bachelors',
    8: 'masters',
    9: 'unknown'
}

# Replace values
df['PB3_15'] = df['PB3_15'].replace(expected_grd)

# Rename the column
df.rename(columns={'PB3_15': 'expected_grade'}, inplace=True)

In [43]:
df.groupby('expected_grade').size()

expected_grade
bachelors      11187
high_school      506
masters         1776
primary           27
secondary        173
tech_bacc        183
tech_school       14
univ_tech        205
unknown          357
dtype: int64

In [44]:
len(df.expected_grade)

19973

In [45]:
df['expected_grade'].isnull().sum()

np.int64(5545)

**Economic Participation**

The PD3_1 value indicates the economic participation of the student to the household

In this case we'll replace numerical values with descriptive codes, to enhance interpretability an simplify model by standarizing formats

| Numeric Value | Description (in Spanish)                                               | Description (Translated)                                      | New Value              |
|---------------|------------------------------------------------------------------------|--------------------------------------------------------------|------------------------|
| 1             | trabajó por lo menos una hora (tenía trabajo pero no trabajó)?         | Worked at least one hour (had a job but didn't work)?         | worked_one_hour       |
| 2             | vendió o hizo algún producto para vender?                              | Sold or made a product to sell?                              | sold_product          |
| 3             | ayudó en las labores del campo, cría de animales, o en el negocio de un familiar o de otra persona? | Helped with farming, animal husbandry, or a family/other's business? | family_business_help |
| 4             | a cambio de un pago realizó otro tipo de actividad? (lavó o planchó ajeno, cuidó niños) | Performed other paid activity? (laundry, ironing, childcare) | paid_other_work       |
| 5             | estuvo de aprendiz o haciendo su servicio social?                      | Was an apprentice or doing community service?                | apprentice_service    |
| 6             | buscó trabajo?                                                        | Searched for a job?                                          | job_search            |
| 7             | Estudia o está en otra situación diferente a las anteriores            | Studying or in a different situation                         | studying_other        |
| b             | No sabe                                                  | Doesn't know                                           | unknown        |


Initial Check


In [46]:
df.groupby('PD3_1').size()

PD3_1
1.0    2299
2.0      72
3.0     430
4.0      19
5.0     115
6.0     158
7.0    6482
dtype: int64

In [47]:
# new value dictionary
economic_part = {
    1: 'worked_one_hour',
    2: 'sold_product',
    3: 'family_business_help',
    4: 'paid_other_work',
    5: 'apprentice_service',
    6: 'job_search',
    7: 'studying_other',
    8: 'unknown'
}

# Replace values
df['PD3_1'] = df['PD3_1'].replace(economic_part)

# Rename the column
df.rename(columns={'PD3_1': 'economic_participation'}, inplace=True)

In [48]:
df.groupby('economic_participation').size()

economic_participation
apprentice_service       115
family_business_help     430
job_search               158
paid_other_work           19
sold_product              72
studying_other          6482
worked_one_hour         2299
dtype: int64

In [49]:
df['economic_participation'].isnull().sum()

np.int64(10398)

**Economic Consequences**

The PD3_3 value indicates the potential economic consequences for the household if the individual stops working or performing their activity.

In this case we'll replace numerical values with descriptive codes, to enhance interpretability an simplify model by standarizing formats



| Numeric Value | Description (in Spanish)                                       | Description (Translated)                                     | New Value              |
|---------------|----------------------------------------------------------------|-------------------------------------------------------------|------------------------|
| 1             | Tendría que contratarse a otra persona en el negocio familiar | Another person would need to be hired for the family business | hire_replacement      |
| 2             | El hogar no podría sostenerse económicamente                  | The household could not sustain itself economically         | economic_unsustain    |
| 3             | El ingreso del hogar se vería disminuido                      | Household income would decrease                             | income_decrease       |
| 4             | La carga de trabajo sería más pesada para los demás           | Workload would increase for others                          | workload_increase     |
| 5             | El hogar tendría que destinar ingresos para los gastos de ella (él) | Household would need to allocate income for her/his expenses | allocate_income       |
| 6             | Ella (él) ya no podría continuar estudiando                   | She/he could no longer continue studying                    | education_stop        |
| 7             | Otra consecuencia                                             | Another consequence                                         | other_consequence     |
| 8             | No habría consecuencia                                        | There would be no consequence                               | no_consequence        |
| b             | Blanco por secuencia                                          | Blank by sequence                                           | unknown        |


Initial check

In [50]:
df['PD3_3'].isnull().sum()

np.int64(17038)

In [51]:
df.groupby('PD3_3').size()

PD3_3
1.0      44
2.0     160
3.0     645
4.0     126
5.0     274
6.0     228
7.0      27
8.0    1431
dtype: int64

In [52]:
# new value dictionary
economic_cons = {
    1: 'hire_replacement',
    2: 'economic_unsustain',
    3: 'income_decrease',
    4: 'workload_increase',
    5: 'allocate_income',
    6: 'education_stop',
    7: 'other_consequence',
    8: 'no_consequence'
}

# Replace values
df['PD3_3'] = df['PD3_3'].replace(economic_cons)

# Rename the column
df.rename(columns={'PD3_3': 'economic_consequences'}, inplace=True)

In [53]:
df.groupby('economic_consequences').size()

economic_consequences
allocate_income        274
economic_unsustain     160
education_stop         228
hire_replacement        44
income_decrease        645
no_consequence        1431
other_consequence       27
workload_increase      126
dtype: int64

### Ensure Correct Type for Boolean features

**Evaluation Methods**

PA3_8_1 to PA3_8_8 indicates wich was the kind of method used to evaluate student's performance.

In this case, we'll use "em" preffix for Evaluation Method

Initial check

In [54]:
df[['PA3_8_1','PA3_8_2','PA3_8_3','PA3_8_4','PA3_8_5','PA3_8_6','PA3_8_7','PA3_8_8']].head(10)

,PA3_8_1,PA3_8_2,PA3_8_3,PA3_8_4,PA3_8_5,PA3_8_6,PA3_8_7,PA3_8_8
0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0
1,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0
2,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0
4,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0
7,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0
8,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0
9,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0
10,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
12,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0
13,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0


In [55]:
columns_to_transform = ['PA3_8_1','PA3_8_2','PA3_8_3','PA3_8_4','PA3_8_5','PA3_8_6','PA3_8_7','PA3_8_8']

In [56]:
# Replace Values: 1 -> True, else -> False
df[columns_to_transform] = df[columns_to_transform] == 1

In [57]:
df[['PA3_8_1','PA3_8_2','PA3_8_3','PA3_8_4','PA3_8_5','PA3_8_6','PA3_8_7','PA3_8_8']].head(10)

,PA3_8_1,PA3_8_2,PA3_8_3,PA3_8_4,PA3_8_5,PA3_8_6,PA3_8_7,PA3_8_8
0,True,True,False,False,True,False,False,False
1,True,True,False,False,True,False,False,False
2,True,True,False,False,False,False,False,False
4,True,True,False,False,True,True,False,False
7,True,True,False,False,False,False,False,False
8,True,True,False,False,False,False,False,False
9,True,True,False,False,False,False,False,False
10,True,False,False,False,False,False,False,False
12,True,False,False,True,True,False,False,False
13,True,True,True,False,False,False,False,False


In [58]:
# Rename

column_rename_map = {
    "PA3_8_1": "em_hw_projects",
    "PA3_8_2": "em_tests",
    "PA3_8_3": "em_multimedia_evidence",
    "PA3_8_4": "em_class_participation",
    "PA3_8_5": "em_class_work",
    "PA3_8_6": "em_class_attendance",
    "PA3_8_7": "em_other",
    "PA3_8_8": "em_no_evaluation"
}

df.rename(columns=column_rename_map, inplace=True)


In [59]:
df[['em_hw_projects','em_tests','em_multimedia_evidence','em_class_participation','em_class_work','em_class_attendance','em_other','em_no_evaluation']].head(10)

,em_hw_projects,em_tests,em_multimedia_evidence,em_class_participation,em_class_work,em_class_attendance,em_other,em_no_evaluation
0,True,True,False,False,True,False,False,False
1,True,True,False,False,True,False,False,False
2,True,True,False,False,False,False,False,False
4,True,True,False,False,True,True,False,False
7,True,True,False,False,False,False,False,False
8,True,True,False,False,False,False,False,False
9,True,True,False,False,False,False,False,False
10,True,False,False,False,False,False,False,False
12,True,False,False,True,True,False,False,False
13,True,True,True,False,False,False,False,False


**Educational Tools Used**

PB3_12_1 to PB3_12_8 indicates the educational tools that the student used.

In this case, we'll use "et" preffix for Educational Tools used

Initial Check

In [60]:
df[['PB3_12_1','PB3_12_2','PB3_12_3','PB3_12_4','PB3_12_5','PB3_12_6','PB3_12_7','PB3_12_8']].head(10)

,PB3_12_1,PB3_12_2,PB3_12_3,PB3_12_4,PB3_12_5,PB3_12_6,PB3_12_7,PB3_12_8
0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0
1,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0
2,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0
4,2.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0
7,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0
8,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0
9,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0
10,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0
12,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0
13,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0


In [61]:
columns_to_transform = ['PB3_12_1','PB3_12_2','PB3_12_3','PB3_12_4','PB3_12_5','PB3_12_6','PB3_12_7','PB3_12_8']

In [62]:
# Replace Values: 1 -> True, else -> False
df[columns_to_transform] = df[columns_to_transform] == 1

In [63]:
df[['PB3_12_1','PB3_12_2','PB3_12_3','PB3_12_4','PB3_12_5','PB3_12_6','PB3_12_7','PB3_12_8']].head(10)

,PB3_12_1,PB3_12_2,PB3_12_3,PB3_12_4,PB3_12_5,PB3_12_6,PB3_12_7,PB3_12_8
0,False,True,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False
2,True,False,False,False,False,True,False,False
4,False,True,False,True,False,True,False,False
7,True,True,False,False,False,True,False,False
8,True,True,False,False,False,True,False,False
9,True,True,False,False,False,True,False,False
10,True,True,False,False,False,True,False,False
12,True,True,False,False,False,True,False,False
13,False,True,False,False,False,True,False,False


In [64]:
# Rename
column_rename_map = {
    "PB3_12_1": "et_smartphone",
    "PB3_12_2": "et_laptop",
    "PB3_12_3": "et_desktop_pc",
    "PB3_12_4": "et_tablet",
    "PB3_12_5": "et_flat_screen",
    "PB3_12_6": "et_didactic_material",
    "PB3_12_7": "et_other",
    "PB3_12_8": "et_none"
}

df.rename(columns=column_rename_map, inplace=True)

In [65]:
df[['et_smartphone','et_laptop','et_desktop_pc','et_tablet','et_flat_screen','et_didactic_material','et_other','et_none']].head(10)

,et_smartphone,et_laptop,et_desktop_pc,et_tablet,et_flat_screen,et_didactic_material,et_other,et_none
0,False,True,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False
2,True,False,False,False,False,True,False,False
4,False,True,False,True,False,True,False,False
7,True,True,False,False,False,True,False,False
8,True,True,False,False,False,True,False,False
9,True,True,False,False,False,True,False,False
10,True,True,False,False,False,True,False,False
12,True,True,False,False,False,True,False,False
13,False,True,False,False,False,True,False,False


**Help Received**

PB3_13_1 to PB3_13_7 indicate the type of relationship of the person who helped the student. In this case, we'll use the prefix 'hr' for Help Received.

Initial Check

In [66]:
df[['PB3_13_1','PB3_13_2','PB3_13_3','PB3_13_4','PB3_13_5','PB3_13_6','PB3_13_7']].head(10)

,PB3_13_1,PB3_13_2,PB3_13_3,PB3_13_4,PB3_13_5,PB3_13_6,PB3_13_7
0,1.0,2.0,2.0,2.0,2.0,2.0,2.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,2.0,2.0,2.0,2.0,2.0,2.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1.0,2.0,2.0,2.0,2.0,2.0,2.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,2.0,2.0,2.0,1.0,2.0,2.0,2.0
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,2.0,2.0,1.0,2.0,2.0,2.0,2.0


In [67]:
columns_to_transform = ['PB3_13_1','PB3_13_2','PB3_13_3','PB3_13_4','PB3_13_5','PB3_13_6','PB3_13_7']

In [68]:
# Replace Values: 1 -> True, else -> False
df[columns_to_transform] = df[columns_to_transform] == 1

In [69]:
df[['PB3_13_1','PB3_13_2','PB3_13_3','PB3_13_4','PB3_13_5','PB3_13_6','PB3_13_7']].head(10)

,PB3_13_1,PB3_13_2,PB3_13_3,PB3_13_4,PB3_13_5,PB3_13_6,PB3_13_7
0,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False
4,False,False,False,False,False,False,False
7,True,False,False,False,False,False,False
8,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False
10,False,False,False,True,False,False,False
12,False,False,False,False,False,False,False
13,False,False,True,False,False,False,False


In [70]:
# Rename
column_rename_map = {
    "PB3_13_1": "hr_mother",
    "PB3_13_2": "hr_father",
    "PB3_13_3": "hr_female_relative",
    "PB3_13_4": "hr_male_relative",
    "PB3_13_5": "hr_female_non_relative",
    "PB3_13_6": "hr_male_non_relative",
    "PB3_13_7": "hr_none"
}

df.rename(columns=column_rename_map, inplace=True)

In [71]:
df[['hr_mother','hr_father','hr_female_relative','hr_male_relative','hr_female_non_relative','hr_male_non_relative','hr_none']].head(10)

,hr_mother,hr_father,hr_female_relative,hr_male_relative,hr_female_non_relative,hr_male_non_relative,hr_none
0,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False
4,False,False,False,False,False,False,False
7,True,False,False,False,False,False,False
8,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False
10,False,False,False,True,False,False,False
12,False,False,False,False,False,False,False
13,False,False,True,False,False,False,False


**Academic Pressure**

PB3_16_1 to PB3_16_5 indicate the level of pressure the student is experiencing regarding academic workloads. In this case, we'll use the prefix 'ap' for Academic Pressure.

Initial check

In [72]:
df[['PB3_16_1','PB3_16_2','PB3_16_3','PB3_16_4','PB3_16_5']].head(10)

,PB3_16_1,PB3_16_2,PB3_16_3,PB3_16_4,PB3_16_5
0,1.0,2.0,2.0,2.0,2.0
1,1.0,2.0,2.0,2.0,2.0
2,1.0,2.0,1.0,2.0,2.0
4,2.0,2.0,2.0,2.0,1.0
7,2.0,2.0,2.0,2.0,1.0
8,1.0,2.0,2.0,2.0,2.0
9,2.0,2.0,2.0,2.0,1.0
10,2.0,2.0,2.0,2.0,1.0
12,1.0,2.0,2.0,2.0,2.0
13,2.0,2.0,2.0,2.0,1.0


In [73]:
columns_to_transform = ['PB3_16_1','PB3_16_2','PB3_16_3','PB3_16_4','PB3_16_5']

In [74]:
# Replace Values: 1 -> True, else -> False
df[columns_to_transform] = df[columns_to_transform] == 1

In [75]:
df[['PB3_16_1','PB3_16_2','PB3_16_3','PB3_16_4','PB3_16_5']].head(10)

,PB3_16_1,PB3_16_2,PB3_16_3,PB3_16_4,PB3_16_5
0,True,False,False,False,False
1,True,False,False,False,False
2,True,False,True,False,False
4,False,False,False,False,True
7,False,False,False,False,True
8,True,False,False,False,False
9,False,False,False,False,True
10,False,False,False,False,True
12,True,False,False,False,False
13,False,False,False,False,True


In [76]:
# Rename
column_rename_map = {
    "PB3_16_1": "ap_stressed",
    "PB3_16_2": "ap_depressed",
    "PB3_16_3": "ap_academic_desperation",
    "PB3_16_4": "ap_social_difficulty",
    "PB3_16_5": "ap_no_issues",
}

df.rename(columns=column_rename_map, inplace=True)

In [77]:
df[['ap_stressed','ap_depressed','ap_academic_desperation','ap_social_difficulty','ap_no_issues']].head(10)

,ap_stressed,ap_depressed,ap_academic_desperation,ap_social_difficulty,ap_no_issues
0,True,False,False,False,False
1,True,False,False,False,False
2,True,False,True,False,False
4,False,False,False,False,True
7,False,False,False,False,True
8,True,False,False,False,False
9,False,False,False,False,True
10,False,False,False,False,True
12,True,False,False,False,False
13,False,False,False,False,True


### Reformat Target Column

Initial check

In [78]:
df['PA3_4'].isnull().sum()

np.int64(0)

In [79]:
df.groupby('PA3_4').size()

PA3_4
1.0    19585
2.0      388
dtype: int64

In [80]:
# Replace Values: 1 -> True, else -> False
df['PA3_4'] = df['PA3_4'] == 1

In [81]:
df.groupby('PA3_4').size()

PA3_4
False      388
True     19585
dtype: int64

In [82]:
# Renombrar la columna 'old_name' a 'new_name'
df.rename(columns={'PA3_4': 'finished_grade'}, inplace=True)

In [83]:
df.groupby('finished_grade').size()

finished_grade
False      388
True     19585
dtype: int64

### Rename the rest of useful columns

In [84]:
# Rename
column_rename_map = {
    "N_REN": "resident_seq_number",
    "EDAD": "age",
    "PB3_14": "help_hours",
    "PD3_2": "work_hours",
    "ENT": "state_number",
}

df.rename(columns=column_rename_map, inplace=True)

### Remove unuseful columns

In [85]:
df.drop(
    columns=[
        'FOLIO',
        'P3_1',
        'P3_2',
        'PA3_1',
        'PA3_5',
        'PA3_6',
        'PA3_7_1',
        'PA3_7_2',
        'PA3_7_3',
        'PB3_1',
        'PB3_2',
        'PB3_3',
        'FILTRO_A',
        'PB3_4',
        'PB3_5_NIVEL',
        'PB3_5_BIMESTRE',
        'PB3_5_TRIMESTRE',
        'PB3_5_CUATRIMESTRE',
        'PB3_5_SEMESTRE',
        'PB3_5_ANIO',
        'PB3_5_MODMAT',
        'PB3_6',
        'PB3_7',
        'PB3_8',
        'PB3_9_1',
        'PB3_9_2',
        'PB3_9_3',
        'PB3_10_1',
        'PB3_10_2',
        'PB3_10_3',
        'PB3_10_4',
        'PB3_10_5',
        'PB3_11_1',
        'PB3_11_2',
        'PB3_11_3',
        'PB3_11_4',
        'PB3_11_5',
        'FILTRO_B',
        'FILTRO_C',
        'PC3_1',
        'PC3_2',
        'PC3_3_1',
        'PC3_3_2',
        'PC3_4',
        'PC3_5',
        'PC3_6',
        'PC3_7',
        'PC3_8',
        'FILTRO_D',
        'NIVEL_A',
        'GRADO_A',
        'NIVEL_B',
        'GRADO_B',
        'ESC',
        'FACTOR'
    ],
    inplace=True
)


In [86]:
df.head()

,resident_seq_number,sex,age,school_type,school_grade,finished_grade,em_hw_projects,em_tests,em_multimedia_evidence,em_class_participation,...,ap_depressed,ap_academic_desperation,ap_social_difficulty,ap_no_issues,economic_participation,work_hours,economic_consequences,state_number,period_type,period_number
0,1,male,13,public,primary,True,True,True,False,False,...,False,False,False,False,NaN,NaN,NaN,24,year,6.0
1,2,female,19,public,bachelors,True,True,True,False,False,...,False,False,False,False,studying_other,NaN,NaN,24,year,1.0
2,1,female,8,public,primary,True,True,True,False,False,...,False,True,False,False,NaN,NaN,NaN,22,year,2.0
4,1,male,27,private,bachelors,True,True,True,False,False,...,False,False,False,True,worked_one_hour,25.0,no_consequence,26,quadrimester,2.0
7,2,male,11,private,primary,True,True,True,False,False,...,False,False,False,True,NaN,NaN,NaN,18,year,5.0


In [87]:
df.dtypes

resident_seq_number          int64
sex                         object
age                          int64
school_type                 object
school_grade                object
finished_grade                bool
em_hw_projects                bool
em_tests                      bool
em_multimedia_evidence        bool
em_class_participation        bool
em_class_work                 bool
em_class_attendance           bool
em_other                      bool
em_no_evaluation              bool
et_smartphone                 bool
et_laptop                     bool
et_desktop_pc                 bool
et_tablet                     bool
et_flat_screen                bool
et_didactic_material          bool
et_other                      bool
et_none                       bool
hr_mother                     bool
hr_father                     bool
hr_female_relative            bool
hr_male_relative              bool
hr_female_non_relative        bool
hr_male_non_relative          bool
hr_none             

In [88]:
df.shape

(19973, 42)

In [89]:
# Export data to a pivot csv file
df.to_csv('enape_db_formated.csv', index=False)